In [1]:
# Installing necessary libraries
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime # for more modules later

In [14]:
# Create a list with all files in the folder 
# change file path
folderpath = r"2022-citibike-tripdata"
filepath = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
# had to use zipfile to open because read_csv can"t read multiple files from one zip
import zipfile

# include all columns
columns = ["ride_id", "started_at", "ended_at", "start_station_name", "end_station_name", "start_lat", "start_lng", "end_lat", "end_lng"]

# create emtpy dataframe for citi bike data 
citi_df = pd.DataFrame()

# go through every zip in the filepath (12 for each month)
for zip_file in filepath:
    # Use zipfile to open each file
    zf = zipfile.ZipFile(zip_file)
    # read files individually
    dfs = [pd.read_csv(zf.open(f), usecols = columns ) for f in zf.namelist()] 
    # combine all files from singular zip file
    df = pd.concat(dfs,ignore_index=True)
    # add to citi_df
    citi_df = pd.concat([citi_df, df], axis=0)
    
citi_df

,ride_id,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng
0,3A2034DA67C8874B,2022-09-14 20:37:39.155,2022-09-14 21:05:39.760,N 12 St & Bedford Ave,Albany St & Greenwich St,40.720796,-73.954882,40.709267,-74.013247
1,F1C63DACDCC1C33D,2022-09-17 19:21:39.407,2022-09-17 20:08:32.670,Union Ave & Jackson St,Metropolitan Ave & Meeker Ave,40.716075,-73.952029,40.714133,-73.952344
2,94372C52777F4AC1,2022-09-17 16:49:21.830,2022-09-17 17:01:51.537,S 4 St & Wythe Ave,St Marks Pl & 1 Ave,40.712874,-73.965935,40.727791,-73.985649
3,44818FEC94B62B66,2022-09-08 12:27:40.019,2022-09-08 12:47:15.649,Amsterdam Ave & W 73 St,Washington St & Gansevoort St,40.779668,-73.980930,40.739323,-74.008119
4,F8A63709F214EBAA,2022-09-16 19:00:19.266,2022-09-16 19:07:28.905,University Pl & E 14 St,Washington St & Gansevoort St,40.734814,-73.992085,40.739323,-74.008119
...,...,...,...,...,...,...,...,...,...
3343909,DB723604F1D6F829,2022-06-03 19:10:22.274,2022-06-03 20:07:05.613,E 23 St & 1 Ave,India St Pier,40.736502,-73.978095,40.731735,-73.961239
3343910,1FC10304BB3FC5F4,2022-06-03 11:51:51.761,2022-06-03 12:08:15.186,Park Ave & E 124 St,E 89 St & York Ave,40.804555,-73.939686,40.777945,-73.946041
3343911,05CEDA5EBA1BD8B6,2022-06-03 18:30:03.832,2022-06-03 20:34:24.150,W 30 St & 8 Ave,7 Ave & Park Pl,40.749385,-73.994580,40.677615,-73.973243
3343912,DAB931393D7F944E,2022-06-16 16:44:09.916,2022-06-16 16:47:52.246,W 87 St & Amsterdam Ave,W 87 St & West End Ave,40.788390,-73.974700,40.789622,-73.977570


My code in described in comments. 

In [4]:
# NOAA token
Token = "UyzWJXDqukWjTLQeXjiHAxHkgloCIgNB" 
# Request New York LaGuardia’s weather station for 2022 
r = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31",headers={"token":Token})
# Sort as json format
weatherJson = json.loads(r.text) 

In [5]:
# Wrangle data from weather

avg_temps = [item for item in weatherJson["results"] if item["datatype"]=="TAVG"]
date_temp = [item["date"] for item in avg_temps]
temps = [item["value"] for item in avg_temps]

# turn into df, fix date and tmep
df_temps = pd.DataFrame({"date":date_temp, "avgTemp":temps})
df_temps["date"] = [datetime.datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in date_temp] 
df_temps["avgTemp"] = [float(v)/10.0 for v in temps]

df_temps.to_csv("NY_LaGuardia_weather-station_2022")

In [6]:
# convert citi_df date format to be the same as df_temps

citi_df["date"] = citi_df["started_at"].apply(lambda x: str(x)[0:10]) 
citi_df["date"] = citi_df["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d")) 

In [7]:
# dropping dates from 2022
year = datetime.datetime.strptime("2022-01-01", "%Y-%m-%d")
citi_df = citi_df.loc[citi_df["date"] >= year]

In [8]:
citi_df.sort_values(by="date") # check, shows than it has none 2021 values

,ride_id,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,date
865846,05D00A80FBBEB287,2022-01-01 14:30:50.287,2022-01-02 16:13:36.951,Pearl St & Peck Slip,NaN,40.708485,-74.002751,NaN,NaN,2022-01-01
271247,B6B79365A0B6642B,2022-01-01 14:33:03.780,2022-01-01 14:39:13.335,Front St & Gold St,Clinton St & Tillary St,40.702240,-73.982578,40.696233,-73.991421,2022-01-01
667497,69D2AA31C704D09A,2022-01-01 13:31:07.449,2022-01-01 13:53:17.405,Central Park West & W 76 St,Madison Ave & E 120 St,40.778968,-73.973747,40.802950,-73.942954,2022-01-01
667525,D06AE0665CF378FB,2022-01-01 07:48:37.764,2022-01-01 07:56:17.630,Henry St & Grand St,E 7 St & Avenue A,40.714211,-73.981095,40.726218,-73.983799,2022-01-01
945139,7C1B090DA6F27F16,2022-01-01 14:30:56.726,2022-01-01 14:45:30.165,North Moore St & Greenwich St,Pearl St & Peck Slip,40.720195,-74.010301,40.708485,-74.002751,2022-01-01
...,...,...,...,...,...,...,...,...,...,...
87816,B2FC766CC1C65110,2022-12-31 11:36:21.712,2022-12-31 11:39:11.707,5 Ave & E 103 St,E 103 St & Lexington Ave,40.792255,-73.952499,40.790305,-73.947558,2022-12-31
925679,C230E6F5514C1312,2022-12-31 16:18:43.585,2022-12-31 16:34:56.372,Banker St & Meserole Ave,Greene Ave & Nostrand Ave,40.726207,-73.956209,40.688334,-73.950916,2022-12-31
532116,E56C325E168AF871,2022-12-31 23:29:44.048,2022-12-31 23:34:28.491,1 Ave & E 78 St,5 Ave & E 78 St,40.771563,-73.953300,40.776321,-73.964274,2022-12-31
925816,E885582AC664A32C,2022-12-31 13:23:30.162,2022-12-31 13:38:31.120,Underhill Ave & Lincoln Pl,Pacific St & Nevins St,40.674012,-73.967146,40.685376,-73.983021,2022-12-31


In [9]:
citi_df.sort_values(by="date") # check
citi_df = citi_df.reset_index()

In [10]:
df_merged = citi_df.merge(df_temps, how = "left", on = "date", indicator = True)
df_merged = df_merged.drop(["index"], axis=1)
df_merged

,ride_id,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,date,avgTemp,_merge
0,3A2034DA67C8874B,2022-09-14 20:37:39.155,2022-09-14 21:05:39.760,N 12 St & Bedford Ave,Albany St & Greenwich St,40.720796,-73.954882,40.709267,-74.013247,2022-09-14,22.9,both
1,F1C63DACDCC1C33D,2022-09-17 19:21:39.407,2022-09-17 20:08:32.670,Union Ave & Jackson St,Metropolitan Ave & Meeker Ave,40.716075,-73.952029,40.714133,-73.952344,2022-09-17,20.8,both
2,94372C52777F4AC1,2022-09-17 16:49:21.830,2022-09-17 17:01:51.537,S 4 St & Wythe Ave,St Marks Pl & 1 Ave,40.712874,-73.965935,40.727791,-73.985649,2022-09-17,20.8,both
3,44818FEC94B62B66,2022-09-08 12:27:40.019,2022-09-08 12:47:15.649,Amsterdam Ave & W 73 St,Washington St & Gansevoort St,40.779668,-73.980930,40.739323,-74.008119,2022-09-08,21.8,both
4,F8A63709F214EBAA,2022-09-16 19:00:19.266,2022-09-16 19:07:28.905,University Pl & E 14 St,Washington St & Gansevoort St,40.734814,-73.992085,40.739323,-74.008119,2022-09-16,19.6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
29838161,DB723604F1D6F829,2022-06-03 19:10:22.274,2022-06-03 20:07:05.613,E 23 St & 1 Ave,India St Pier,40.736502,-73.978095,40.731735,-73.961239,2022-06-03,19.5,both
29838162,1FC10304BB3FC5F4,2022-06-03 11:51:51.761,2022-06-03 12:08:15.186,Park Ave & E 124 St,E 89 St & York Ave,40.804555,-73.939686,40.777945,-73.946041,2022-06-03,19.5,both
29838163,05CEDA5EBA1BD8B6,2022-06-03 18:30:03.832,2022-06-03 20:34:24.150,W 30 St & 8 Ave,7 Ave & Park Pl,40.749385,-73.994580,40.677615,-73.973243,2022-06-03,19.5,both
29838164,DAB931393D7F944E,2022-06-16 16:44:09.916,2022-06-16 16:47:52.246,W 87 St & Amsterdam Ave,W 87 St & West End Ave,40.788390,-73.974700,40.789622,-73.977570,2022-06-16,20.3,both


In [11]:
df_merged.shape

(29838166, 12)

In [12]:
df_merged['_merge'].value_counts(dropna = False)

_merge
both          29838166
left_only            0
right_only           0
Name: count, dtype: int64

In [13]:
df_merged.to_csv("Citi_Bike_LG_merge")